In [1]:
import numpy as np
import pandas as pd
from tree import decisionTreeNode
import random

In [2]:
dat = pd.read_csv('../data/dat1.csv')

In [117]:
ff = np.where(dat.Class == 1)
print(len(dat.index))
print(ff)


142404
(array([   541,    623,   4920,   6108,   6329,   6331,   6334,   6336,
         6338,   6427,   6446,   6472,   6529,   6609,   6641,   6717,
         6719,   6734,   6774,   6820,   6870,   6882,   6899,   6903,
         6971,   8296,   8312,   8335,   8615,   8617,   8842,   8845,
         8972,   9035,   9179,   9252,   9487,   9509,  10204,  10484,
        10497,  10498,  10568,  10630,  10690,  10801,  10891,  10897,
        11343,  11710,  11841,  11880,  12070,  12108,  12261,  12369,
        14104,  14170,  14197,  14211,  14338,  15166,  15204,  15225,
        15451,  15476,  15506,  15539,  15566,  15736,  15751,  15781,
        15810,  16415,  16780,  16863,  17317,  17366,  17407,  17453,
        17480,  18466,  18472,  18773,  18809,  20198,  23308,  23422,
        26802,  27362,  27627,  27738,  27749,  29687,  30100,  30314,
        30384,  30398,  30442,  30473,  30496,  31002,  33276,  39183,
        40085,  40525,  41395,  41569,  41943,  42007,  42009,  42473

In [118]:
n = 20000
nn = 40525
dat_train = dat.loc[:n, :]
dat_test = dat.loc[nn, :]

In [119]:
X = dat_train.drop('Class', axis=1)
e=0
l=np.log2(len(X.index))
# X

In [120]:
def nodeInfo(X, Length=0, isLeaf=False, splitAtt=None, splitValue=None, isLeftChild=None, isRoot=True, Amount=0):
    node = decisionTreeNode()
    node.setLength(Length)
    node.setsplitAtt(splitAtt)
    node.setsplitValue(splitValue)
    node.setisLeftChild(isLeftChild)
    node.setisRoot(isRoot)
    node.setAmount(Amount)
    if(isLeaf):
        node.setisLeaf()
    return(node)

In [121]:
def iTree(X, e, l, Length=0, isLeaf=False, splitAtt=None, splitValue=None, isLeftChild=None, isRoot=True, Amount=0):
    if(e < l and len(X.index) > 1):
        node = nodeInfo(X, Length, isLeaf, splitAtt, splitValue, isLeftChild, isRoot, Amount)
        splitAtt = X.columns[random.randint(0, (len(X.columns)-1))]
        minimum = min(X[splitAtt])
        maximum = max(X[splitAtt])
        while(minimum == maximum):
            splitAtt = X.columns[random.randint(0, (len(X.columns)-1))]
            minimum = min(X[splitAtt])
            maximum = max(X[splitAtt])
        splitValue = np.random.uniform(low=minimum, high=maximum)
        Xl = X.loc[X[splitAtt]<=splitValue, :]
        Xr = X.loc[X[splitAtt]>splitValue, :]
        ee = e + 1
        Amountl = len(Xl.index)
        Child_left = iTree(X=Xl, e=ee, l=l, Length=ee, splitAtt=splitAtt, splitValue=splitValue, 
                           isLeftChild=True, isRoot=False, Amount=Amountl)
        node.setChildren(Child_left)
        Amountr = len(Xr.index)
        Child_right = iTree(X=Xr, e=ee, l=l, Length=ee, splitAtt=splitAtt, splitValue=splitValue, 
                            isLeftChild=False, isRoot=False, Amount=Amountr)
        node.setChildren(Child_right)
    else:
        Amounta = len(X.index)
        node = nodeInfo(X, Length, isLeaf=True, splitAtt=splitAtt, splitValue=splitValue, 
                        isLeftChild=isLeftChild, isRoot=isRoot, Amount=Amounta)
    return(node)
        
        

In [122]:
def printTree(node, depth = 0):
    if not node.getisRoot():
        x = node.getsplitAtt()
        if node.getisLeftChild():
            equality = "<="
        else:
            equality = ">"
        threshold = "%.6f" % node.getsplitValue() # Six decimal
        if node.getisLeaf():
            print(depth * "|\t" + "%s %s %s [%d]: %s" \
                                  % (node.getsplitAtt(), equality, threshold,
                                     node.getAmount(), node.getLength()))
        else:
            print(depth * "|\t" + "%s %s %s [%d]" \
                                  % (node.getsplitAtt(), equality, threshold,
                                     node.getAmount()))
        depth +=1
    for child in node.getChildren():
        printTree(child, depth)

In [123]:
%time mytree = iTree(X,e,l)

CPU times: user 414 ms, sys: 33.9 ms, total: 448 ms
Wall time: 449 ms


In [124]:
# printTree(mytree)

In [125]:
l

14.287784512498185

In [126]:
def classify(X, decisionTree):
    prediction = None
    if decisionTree.getisLeaf():
        return(decisionTree.getLength() + decisionTree.getAmount())
    for child in decisionTree.getChildren():
        splitAtt = child.getsplitAtt()
        splitValue = child.getsplitValue()
        if child.getisLeftChild():
            if X[splitAtt] <= splitValue:
                prediction = classify(X, child)
        else:
            if X[splitAtt] > splitValue:
                prediction = classify(X, child)
    return(prediction)
        

In [129]:
def predict(train, test, decisionTree):
    numData = len(train.index)
    prediction = np.zeros(numData)
    for i in range(numData):
        instance = train.loc[i, :]
        prediction[i] = classify(instance, decisionTree)
    pred_y = classify(test, decisionTree)
    return prediction, pred_y

In [130]:
%time prediction, pred_y = predict(dat_train, dat_test, mytree)

CPU times: user 9.23 s, sys: 34.3 ms, total: 9.27 s
Wall time: 9.44 s


In [133]:
sum(prediction <= pred_y)/len(prediction)

0.0143992800359982

In [132]:
sum(dat_train.Class)/len(dat_train.index)

0.004249787510624469

In [29]:
node=mytree
node.getisRoot()
child=node.getChildren()[0]
node=child
node.getisRoot()
node.getsplitAtt()
node.getLength()
node.getisLeftChild()

2.584962500721156

In [8]:
node = nodeInfo(X)
splitAtt = X.columns[random.randint(0, (len(X.columns)-1))]
minimum = min(X[splitAtt])
maximum = max(X[splitAtt])
while(minimum == maximum):
    splitAtt = X.columns[random.randint(0, (len(X.columns)-1))]
    minimum = min(X[splitAtt])
    maximum = max(X[splitAtt])
splitValue = np.random.uniform(low=minimum, high=maximum)
Xl = X.loc[X[splitAtt]<=splitValue, :]
Xr = X.loc[X[splitAtt]>splitValue, :]
ee = e + 1
node.setLength(e)
node.setsplitAtt(splitAtt)
node.setsplitValue(splitValue)

In [14]:
Xr

,Time,V1,V2,V10,Amount
0,0.0,-1.359807,-0.072781,0.090794,149.62
2,1.0,-1.358354,-1.340163,0.207643,378.66
3,1.0,-0.966272,-0.185226,-0.054952,123.50


In [15]:
X, e, l=Xr,ee,l

In [16]:
e < l and len(X.index) > 1

True

In [17]:
nodeInfo(X, Length=e, isLeaf=True)

In [16]:
e

1